# SpotipyIterator

An idea for iterable interface I am developing as a contribution to the plamere/spotipy (a Spotify API client)
repository on Github.

In [127]:
import re

import inflect
import json
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials


class SpotipyIterator:
    """Make an iterator to use with Spotify API results"""

    INFLECT = inflect.engine()
    SEARCH_REGEXP = re.compile('^https?://[^/]+/v1/search\\b')
    SEARCH_MAX_OFFSET = 1000

    def __init__(self, *args, **kwargs):
        self._client = None
        if kwargs:
            if 'client' in kwargs:
                self._client = kwargs['client']
                del kwargs['client']
        if not self._client:
            self._client = Spotify(auth_manager=SpotifyClientCredentials(), 
                                   requests_session=True)
        self._collection = None
        if kwargs:
            if 'collection' in kwargs:
                self._collection = kwargs['collection']
                del kwargs['collection']
            else:
                if 'type' in kwargs:
                    self._collection = self.INFLECT.plural(kwargs['type'])
        result, *args = args
        if callable(result):
            result = result(self._client, *args, **kwargs)
        if result and 'total' not in result:
            if not self._collection:
                self._collection, *_ = result.keys()
        self._prepare_next_page(result)

    def __iter__(self):
        return self

    def __next__(self):
        if self._row >= self._page_size:
            if self._page and 'next' in self._page and self._page['next']:
                if self.SEARCH_REGEXP.match(self._page['next']):
                    if self._page['offset'] + self._page['limit'] >= self.SEARCH_MAX_OFFSET:
                        raise StopIteration
                self._prepare_next_page(self._client.next(self._page))
                if self._row >= self._page_size:
                    raise StopIteration
            else:
                raise StopIteration
        this_row = self._row
        self._row = this_row + 1
        return self._page['items'][this_row]

    def _prepare_next_page(self, result):
        if self._collection:
            self._page = result[self._collection]
        else:
            self._page = result
        self._page_size = 0
        if self._page and 'items' in self._page:
            self._page_size = len(self._page['items'])
        self._row = 0
        return self
    

In [128]:
import json
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials

# Create a spotipy.Spotify API client...
sp = Spotify(auth_manager=SpotifyClientCredentials(),
            requests_session=True)
# Have the client search for artists within the prog rock genre, whose
# name contains "Rush"...
name, genre = ('Rush', 'progressive rock')
search = sp.search(f'artist:"{name}" genre:"{genre}"',
                   type='artist')
# Create an iterator from the search results. Unlike other end-points, a
# search organises data into subcollections whose keys will be the plural
# of the type(s) of data requested. So, for searches, we can specify which
# collection of data over which to iterate...
artists = SpotipyIterator(search, collection='artists', client=sp)
print(json.dumps([*artists], indent=True))

[
 {
  "external_urls": {
   "spotify": "https://open.spotify.com/artist/2Hkut4rAAyrQxRdof7FVJq"
  },
  "followers": {
   "href": null,
   "total": 1849016
  },
  "genres": [
   "album rock",
   "art rock",
   "canadian metal",
   "classic canadian rock",
   "classic rock",
   "hard rock",
   "metal",
   "progressive rock",
   "rock"
  ],
  "href": "https://api.spotify.com/v1/artists/2Hkut4rAAyrQxRdof7FVJq",
  "id": "2Hkut4rAAyrQxRdof7FVJq",
  "images": [
   {
    "height": 640,
    "url": "https://i.scdn.co/image/ab6761610000e5eb896c4b043fb3063178afd716",
    "width": 640
   },
   {
    "height": 320,
    "url": "https://i.scdn.co/image/ab67616100005174896c4b043fb3063178afd716",
    "width": 320
   },
   {
    "height": 160,
    "url": "https://i.scdn.co/image/ab6761610000f178896c4b043fb3063178afd716",
    "width": 160
   }
  ],
  "name": "Rush",
  "popularity": 70,
  "type": "artist",
  "uri": "spotify:artist:2Hkut4rAAyrQxRdof7FVJq"
 }
]


In [129]:
# For searches, we may use 'type' (expressed in singular form) to
# identify the type of data over which to iterate. This should be the
# same as the 'type' specified in the call to the search method.
artists = SpotipyIterator(search, type='artist', client=sp)
print(json.dumps([*artists], indent=True))

[
 {
  "external_urls": {
   "spotify": "https://open.spotify.com/artist/2Hkut4rAAyrQxRdof7FVJq"
  },
  "followers": {
   "href": null,
   "total": 1849016
  },
  "genres": [
   "album rock",
   "art rock",
   "canadian metal",
   "classic canadian rock",
   "classic rock",
   "hard rock",
   "metal",
   "progressive rock",
   "rock"
  ],
  "href": "https://api.spotify.com/v1/artists/2Hkut4rAAyrQxRdof7FVJq",
  "id": "2Hkut4rAAyrQxRdof7FVJq",
  "images": [
   {
    "height": 640,
    "url": "https://i.scdn.co/image/ab6761610000e5eb896c4b043fb3063178afd716",
    "width": 640
   },
   {
    "height": 320,
    "url": "https://i.scdn.co/image/ab67616100005174896c4b043fb3063178afd716",
    "width": 320
   },
   {
    "height": 160,
    "url": "https://i.scdn.co/image/ab6761610000f178896c4b043fb3063178afd716",
    "width": 160
   }
  ],
  "name": "Rush",
  "popularity": 70,
  "type": "artist",
  "uri": "spotify:artist:2Hkut4rAAyrQxRdof7FVJq"
 }
]


In [130]:
# But the iterator is also smart enough to the first collection it
# finds for a search result. So you can also be lazy...
artists = SpotipyIterator(search, client=sp)
print(json.dumps([*artists], indent=True))

[
 {
  "external_urls": {
   "spotify": "https://open.spotify.com/artist/2Hkut4rAAyrQxRdof7FVJq"
  },
  "followers": {
   "href": null,
   "total": 1849016
  },
  "genres": [
   "album rock",
   "art rock",
   "canadian metal",
   "classic canadian rock",
   "classic rock",
   "hard rock",
   "metal",
   "progressive rock",
   "rock"
  ],
  "href": "https://api.spotify.com/v1/artists/2Hkut4rAAyrQxRdof7FVJq",
  "id": "2Hkut4rAAyrQxRdof7FVJq",
  "images": [
   {
    "height": 640,
    "url": "https://i.scdn.co/image/ab6761610000e5eb896c4b043fb3063178afd716",
    "width": 640
   },
   {
    "height": 320,
    "url": "https://i.scdn.co/image/ab67616100005174896c4b043fb3063178afd716",
    "width": 320
   },
   {
    "height": 160,
    "url": "https://i.scdn.co/image/ab6761610000f178896c4b043fb3063178afd716",
    "width": 160
   }
  ],
  "name": "Rush",
  "popularity": 70,
  "type": "artist",
  "uri": "spotify:artist:2Hkut4rAAyrQxRdof7FVJq"
 }
]


In [131]:
# In fact, if you don't provide a 'client', the iterator will create
# and use it's own, allowing laziness to be taken to a new level...
artists = SpotipyIterator(sp.search(f'artist:"{name}" genre:"{genre}"',
                                    type='artist'))
print(json.dumps([*artists], indent=True))

[
 {
  "external_urls": {
   "spotify": "https://open.spotify.com/artist/2Hkut4rAAyrQxRdof7FVJq"
  },
  "followers": {
   "href": null,
   "total": 1849016
  },
  "genres": [
   "album rock",
   "art rock",
   "canadian metal",
   "classic canadian rock",
   "classic rock",
   "hard rock",
   "metal",
   "progressive rock",
   "rock"
  ],
  "href": "https://api.spotify.com/v1/artists/2Hkut4rAAyrQxRdof7FVJq",
  "id": "2Hkut4rAAyrQxRdof7FVJq",
  "images": [
   {
    "height": 640,
    "url": "https://i.scdn.co/image/ab6761610000e5eb896c4b043fb3063178afd716",
    "width": 640
   },
   {
    "height": 320,
    "url": "https://i.scdn.co/image/ab67616100005174896c4b043fb3063178afd716",
    "width": 320
   },
   {
    "height": 160,
    "url": "https://i.scdn.co/image/ab6761610000f178896c4b043fb3063178afd716",
    "width": 160
   }
  ],
  "name": "Rush",
  "popularity": 70,
  "type": "artist",
  "uri": "spotify:artist:2Hkut4rAAyrQxRdof7FVJq"
 }
]


In [132]:
# ... but, this isn't ideal because, using the example above, we created
# and used our own instance of the spotipy.Spotify API client to get the
# first page of search results, but the iterator had to create another
# to fetch any remaining results. 
#
# There is another way to use the iterator that eliminates this kind of
# waste. Instead of passing the first page of results, you may pass a 
# function and follow it with the arguments that the iterator can pass
# to it when it is called. The iterator will then create and use the same
# client for all pages of results. You can still pass in your own client
# if you need to create it a specific way, but you don't need to.
#
# A side-benefit here, in the case of Spotify.search, is that the iterator
# is able to infer the 'type' of data being iterated over from the list of
# arguments passed to the function it will call. There is no requirement
# to specify the 'type' again...
artists = SpotipyIterator(Spotify.search,
                          f'artist:"{name}" genre:"{genre}"',
                          type='artist')
print(json.dumps([*artists], indent=True))

[
 {
  "external_urls": {
   "spotify": "https://open.spotify.com/artist/2Hkut4rAAyrQxRdof7FVJq"
  },
  "followers": {
   "href": null,
   "total": 1849016
  },
  "genres": [
   "album rock",
   "art rock",
   "canadian metal",
   "classic canadian rock",
   "classic rock",
   "hard rock",
   "metal",
   "progressive rock",
   "rock"
  ],
  "href": "https://api.spotify.com/v1/artists/2Hkut4rAAyrQxRdof7FVJq",
  "id": "2Hkut4rAAyrQxRdof7FVJq",
  "images": [
   {
    "height": 640,
    "url": "https://i.scdn.co/image/ab6761610000e5eb896c4b043fb3063178afd716",
    "width": 640
   },
   {
    "height": 320,
    "url": "https://i.scdn.co/image/ab67616100005174896c4b043fb3063178afd716",
    "width": 320
   },
   {
    "height": 160,
    "url": "https://i.scdn.co/image/ab6761610000f178896c4b043fb3063178afd716",
    "width": 160
   }
  ],
  "name": "Rush",
  "popularity": 70,
  "type": "artist",
  "uri": "spotify:artist:2Hkut4rAAyrQxRdof7FVJq"
 }
]


In [133]:
# Every example so far has yielded a single result, but we can 
# make the genre a little more broad to show multiple results...
artists = SpotipyIterator(Spotify.search,
                          f'artist:"{name}" genre:"rock"',
                          type='artist')
print(json.dumps([*artists], indent=True))

[
 {
  "external_urls": {
   "spotify": "https://open.spotify.com/artist/2Hkut4rAAyrQxRdof7FVJq"
  },
  "followers": {
   "href": null,
   "total": 1849016
  },
  "genres": [
   "album rock",
   "art rock",
   "canadian metal",
   "classic canadian rock",
   "classic rock",
   "hard rock",
   "metal",
   "progressive rock",
   "rock"
  ],
  "href": "https://api.spotify.com/v1/artists/2Hkut4rAAyrQxRdof7FVJq",
  "id": "2Hkut4rAAyrQxRdof7FVJq",
  "images": [
   {
    "height": 640,
    "url": "https://i.scdn.co/image/ab6761610000e5eb896c4b043fb3063178afd716",
    "width": 640
   },
   {
    "height": 320,
    "url": "https://i.scdn.co/image/ab67616100005174896c4b043fb3063178afd716",
    "width": 320
   },
   {
    "height": 160,
    "url": "https://i.scdn.co/image/ab6761610000f178896c4b043fb3063178afd716",
    "width": 160
   }
  ],
  "name": "Rush",
  "popularity": 70,
  "type": "artist",
  "uri": "spotify:artist:2Hkut4rAAyrQxRdof7FVJq"
 },
 {
  "external_urls": {
   "spotify": "https://

In [134]:
from more_itertools import first_true

# Let's do a bit more with the API and with the SpotifyIterator,
# by listing names of albums released by the artist...
artists = SpotipyIterator(Spotify.search,
                          f'artist:"{name}" genre:"rock"',
                          type='artist')
artist = first_true(artists, None,
                    lambda x: name == x['name'] and 
                              genre in x['genres'])
albums = SpotipyIterator(Spotify.artist_albums,
                         artist['uri'],
                         album_type='album',
                         country='GB')
# TODO There is a problem, here...
#      We are not getting all items from the artist_albums
#      end-point. Suspect there's a snafu with the processing
#      of subsequent pages of results for non-search end-points.
#
album_names = [ 
    x['name'] for x in albums 
                    if '(' not in x['name'] and 
                       'Live' not in x['name']
]
print(json.dumps(album_names, indent=True))
# print(json.dumps([*albums], indent=True))

[
 "Clockwork Angels Tour",
 "Clockwork Angels",
 "Snakes & Arrows",
 "R30",
 "Feedback",
 "Different Stages",
 "Presto",
 "A Show Of Hands",
 "Hold Your Fire",
 "Power Windows",
 "Grace Under Pressure",
 "Signals",
 "Exit ... Stage Left",
 "Permanent Waves",
 "Hemispheres",
 "A Farewell To Kings",
 "All The World's A Stage",
 "2112",
 "Caress Of Steel",
 "Fly By Night",
 "Rush",
 "The First Us Tours"
]
